In [1]:
import os
import scipy.io
import numpy as np
import random
import pandas as pd
import torch
from Utils import compute_KNN_graph, fisher_z_transform, create_graph, to_tensor, create_graph_sliding_window
from sklearn.model_selection import train_test_split
from torch_geometric.data import InMemoryDataset, Data
from torch_geometric.utils import dense_to_sparse
import torch.nn.functional as func
import torch.optim as optim
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.nn import GATConv, ChebConv
import torch.nn as nn
from collections import Counter
import os.path as osp
import csv
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, recall_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit

In [2]:
seed=89
atlas_name= "AAL"
dataset_name = "MDDvHC"


if atlas_name == "AAL":
    start = 0
    end = 116
elif atlas_name == "Craddock":
    start = 228
    end = 428
elif atlas_name == "Dosenbach":
    start = 1408
    end = 1568
else:
    exit()

In [3]:


# Function to set seed
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True)

# Environment variables for reproducibility
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Set your seed
set_seed(seed)

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)


In [4]:
from sklearn.preprocessing import StandardScaler

def normalize(matrix):
    scaler = StandardScaler()
    normalized_matrix = scaler.fit_transform(matrix)
    return normalized_matrix

In [5]:
#sliding window
def create_graph_sliding_window_demographics(X, D, Y, start, end, region=True):
    S = 30 # Sliding Step
    T = 60 # Window Size

    X_adjgraph=[]
    X_featgraph = []
    Y_list = []
    num_samples_per_subject = []

    for i in range(len(Y)):
        #select rows according to atlas
        # bold_matrix = X[i][start:end,:]
        bold_matrix = X[i]
        n = bold_matrix.shape[0]
        demog = D[i]
        # Unsqueeze and expand the demog array to create a 116x4 matrix
        demog_expanded = np.expand_dims(demog, axis=0)  # Shape (1, 4)
        demog_expanded = np.repeat(demog_expanded, n, axis=0)  # Shape (116, 4)
        
        temp_y = Y[i]

        num_rows, num_cols = bold_matrix.shape
        num_samples = 0
        for start_idx in range(0, num_cols - T + 1, S):
            end_idx = start_idx + T
            if end_idx <= num_cols:
                
                if region == True:
                    window_data =  bold_matrix[:, start_idx:end_idx]    #RxR
                else:
                    window_data =  np.transpose(bold_matrix[:, start_idx:end_idx])  #TxT

                window_data1 = np.corrcoef(window_data)
                correlation_matrix_fisher = fisher_z_transform(window_data1)
                correlation_matrix_fisher = np.around(correlation_matrix_fisher, 8)     #upto 8 decimal points
                # Concatenate the demog_expanded with correlation_matrix_fisher along the column
                result_matrix = np.concatenate((correlation_matrix_fisher, demog_expanded), axis=1)

                
                knn_graph = compute_KNN_graph(correlation_matrix_fisher)

                if region == True:
                    X_featgraph.append(result_matrix)
                else:
                    X_featgraph.append(window_data)
                    
                X_adjgraph.append(knn_graph)
                Y_list.append(temp_y)
                num_samples = num_samples+1
            
        num_samples_per_subject.append(num_samples)


    return X_featgraph, X_adjgraph, Y_list, num_samples_per_subject

In [6]:
def create_graph_demographics(X, D, Y, start, end, region=True):
    X_adjgraph=[]
    X_featgraph = []

    for i in range(len(Y)):
        if region == True:
            # bold_matrix = X[i][start:end,:] #RxR
            bold_matrix = X[i]
        else:
            bold_matrix = np.transpose(X[i][start:end,:]) #TxT

        n= bold_matrix.shape[0]
        # Unsqueeze and expand the demog array to create a 116x4 matrix
        demog_expanded = np.expand_dims(D[i], axis=0)  # Shape (1, 4)
        demog_expanded = np.repeat(demog_expanded, n, axis=0)  # Shape (116, 4)
        
        window_data1 = np.corrcoef(bold_matrix)
        correlation_matrix_fisher = fisher_z_transform(window_data1)
        correlation_matrix_fisher = np.around(correlation_matrix_fisher, 8)
        knn_graph = compute_KNN_graph(correlation_matrix_fisher)
        # Concatenate the demog_expanded with correlation_matrix_fisher along the column
        result_matrix = np.concatenate((correlation_matrix_fisher, demog_expanded), axis=1)

        if region == True:
            X_featgraph.append(result_matrix)
        else:
            X_featgraph.append(bold_matrix)
            
        X_adjgraph.append(knn_graph)

    return X_featgraph, X_adjgraph, Y

In [7]:
#final model 


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import ChebConv, global_mean_pool,GATConv



class SkipConnModel(nn.Module):
    def __init__(self, num_features_R, num_classes, k_order, dropout_prob=0.5):
        super(SkipConnModel, self).__init__()
        self.dropout_prob = dropout_prob
        
        self.num_layers = 6
        self.convs = nn.ModuleList()  # List for model 1
        self.bns = nn.ModuleList()    # BatchNorm for model 1
        
        # Define the first GCN model
        self.convs.append(ChebConv(num_features_R, 128, K=3, normalization='sym'))
        self.bns.append(nn.BatchNorm1d(128))
        
        self.convs.append(ChebConv(128, 128, K=3, normalization='sym'))
        self.bns.append(nn.BatchNorm1d(128))

        self.convs.append(ChebConv(128, 128, K=3, normalization='sym'))
        self.bns.append(nn.BatchNorm1d(128))
        
        # self.convs.append(ChebConv(128, 128, K=3, normalization='sym'))
        # self.bns.append(nn.BatchNorm1d(128))
        
        self.out_fc = nn.Linear(128, num_classes)
        self.weights = torch.nn.Parameter(torch.randn(len(self.convs)))
        # Add the decov weight as a parameter
        #self.current_decov_weight = nn.Parameter(torch.tensor(1.0))


    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()
        self.out_fc.reset_parameters()
        torch.nn.init.normal_(self.weights)

    def forward(self, data_R):
        # First GCN model
        x1, edge_index1, edge_attr1 = data_R.x, data_R.edge_index, data_R.edge_attr
        batch1 = data_R.batch
        
        layer_out1 = []  # List to store the outputs of model 1
        x1 = self.convs[0](x1, edge_index1, edge_attr1)
        x1 = self.bns[0](x1)
        x1 = F.relu(x1, inplace=True)
        layer_out1.append(x1)
        x1 = F.dropout(x1, p=self.dropout_prob, training=self.training)
        
                          
        x1 = self.convs[1](x1, edge_index1, edge_attr1)
        x1 = self.bns[1](x1)
        x1 = F.relu(x1, inplace=True)
        x1 = x1 + 0.8 * layer_out1[0]
        layer_out1.append(x1)
        x1 = F.dropout(x1, p=self.dropout_prob, training=self.training) 
        
                          
        x1 = self.convs[2](x1, edge_index1, edge_attr1)
        x1 = self.bns[2](x1)
        x1 = F.relu(x1, inplace=True)
        x1 = x1 + 0.8 * layer_out1[1]
        layer_out1.append(x1)
        # x1 = F.dropout(x1, p=self.dropout_prob, training=self.training) 


        # x1 = self.convs[3](x1, edge_index1, edge_attr1)
        # x1 = self.bns[3](x1)
        # x1 = F.relu(x1, inplace=True)
        # x1 = x1 + 1 * layer_out1[2]
        # layer_out1.append(x1)
        #x1 = F.dropout(x1, p=self.dropout_prob, training=self.training) 
        

        # Apply softmax to the weights
        weight = F.softmax(self.weights, dim=0)
        
        # Multiply each layer output by its corresponding weight
        weighted_outs = [layer_out1[i] * weight[i] for i in range(len(layer_out1))]
        
        # Sum the weighted outputs to get the final embedding
        emb = sum(weighted_outs)
        
        # Apply global mean pooling on the summed weighted outputs
        pooled_emb = global_mean_pool(emb, batch1)

        # Pass the final pooled embedding through the linear layer
        x = self.out_fc(pooled_emb)
        
        return x, pooled_emb


In [8]:

import torch

def DECOV(embeddings):
    # Transpose the embeddings
    embeddings_t = embeddings.T  # Now it's 32 x batch

    # Compute the covariance matrix along the columns
    C = torch.cov(embeddings_t)  # Only works with PyTorch 1.9.0 or later

    # Frobenius norm of the covariance matrix
    C_fro_norm = torch.norm(C, p='fro')

    # Diagonal elements of the covariance matrix
    diag_elements = torch.diag(C,0)

    # L2 norm of the diagonal elements
    C2_l2norm_diag = torch.norm(diag_elements)

    # DeCov loss
    L_DECOV = (C_fro_norm ** 2) - (C2_l2norm_diag ** 2)

    return L_DECOV




In [9]:
def GCN_train(loader):
    model.train()

    pred = []
    label = []
    loss_all = 0

    for data in loader:
        data = data.to(device)
        optimizer.zero_grad()
        output, pooled = model(data)  # Get both output and embeddings

         #Ensure pooled_emb is on the correct device for DECOV
        output = output.to('cpu')
        loss_decov = DECOV(output)
        loss_decov = loss_decov.to(device)  # Move loss_decov back to the GPU if needed

        output = output.to(device)  # Move back to the original device if needed
        
        loss1 = func.cross_entropy(output, data.y) 
        #print("\n\nThe DeCov loss is", loss_decov)
       # print("\n\nThe Cross Entropy loss is", loss1)
        #print("\n\n decov weight",  model.current_decov_weight)

        loss = loss1 + 1e-12*loss_decov
        #loss=loss1
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()

        pred.append(func.softmax(output, dim=1).max(dim=1)[1])
        label.append(data.y)
   
    y_pred = torch.cat(pred, dim=0).cpu().detach().numpy()
    y_true = torch.cat(label, dim=0).cpu().detach().numpy()
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    epoch_sen = tp / (tp + fn)
    epoch_spe = tn / (tn + fp)
    epoch_acc = (tn + tp) / (tn + tp + fn + fp)
    f1 = f1_score(y_true, y_pred)

    return epoch_sen, epoch_spe, epoch_acc, f1, loss_all/len(loader)


def GCN_test(loader):
    model.eval()

    pred = []
    scores = []
    label = []
    loss_all = 0
    for data in loader:
        data = data.to(device)
        output , pooled= model(data)
        
        output = output.to('cpu')
        loss_decov = DECOV(output)
        loss_decov = loss_decov.to(device)  # Move loss_decov back to the GPU if needed

        output = output.to(device)  # Move back to the original device if needed
        
        loss1 = func.cross_entropy(output, data.y)
        loss = loss1 + 1e-12*loss_decov
        loss_all += data.num_graphs * loss.item()
   
        softmax_output = func.softmax(output, dim=1)
        scores.append(softmax_output[:, 1])  # Collect the scores for the positive class
        pred.append(softmax_output.max(dim=1)[1])
        label.append(data.y)

    y_pred = torch.cat(pred, dim=0).cpu().detach().numpy()
    y_scores = torch.cat(scores, dim=0).cpu().detach().numpy()
    y_true = torch.cat(label, dim=0).cpu().detach().numpy()
    
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    epoch_sen = tp / (tp + fn)
    epoch_spe = tn / (tn + fp)
    epoch_acc = (tn + tp) / (tn + tp + fn + fp)
    epoch_f1 = f1_score(y_true, y_pred)
    epoch_auc = roc_auc_score(y_true, y_scores)  # Use scores instead of predicted labels
    return epoch_sen, epoch_spe, epoch_acc, epoch_f1, epoch_auc,loss_all / len(loader)



In [10]:
X_new =  np.load(f'{dataset_name}/{atlas_name}/X.npz')
X_loaded = [X_new[key] for key in X_new.files]
X_loaded = [normalize(matrix) for matrix in X_loaded]

print(X_loaded[0].shape)

Y_loaded = np.load(f'{dataset_name}/{atlas_name}/Y.npy')
print(Y_loaded)

# Load the CSV file
demographic_data = pd.read_csv(f'{dataset_name}/{atlas_name}/demographics_data.csv')

# Convert demographic data to tensor
# Assuming the columns are ['subject_id', 'age', 'edu', 'motion', 'sex']
demographics = demographic_data[['Age', 'Edu','Sex']].values

# sites_labels = np.load(f'{dataset_name}/site_labels.npy')
# print(sites_labels)

(116, 200)
[1 1 1 ... 0 0 0]


In [11]:
# Initialize lists to store segregated data
X_male, Y_male = [], []
X_female, Y_female = [], []
demographics_male, demographics_female = [], []

# Segregate data based on sex
for i, sex in enumerate(demographic_data['Sex']):
    if sex == 0:  # Male
        X_male.append(X_loaded[i])
        Y_male.append(Y_loaded[i])
        demographics_male.append(demographics[i])
    elif sex == 1:  # Female
        X_female.append(X_loaded[i])
        Y_female.append(Y_loaded[i])
        demographics_female.append(demographics[i])

# Convert lists to numpy arrays
Y_male = np.array(Y_male)
Y_female = np.array(Y_female)
demographics_male = np.array(demographics_male)
demographics_female = np.array(demographics_female)

# Print the lengths of the segregated data
print(len(X_male))
print(len(Y_male))
print(len(X_female))
print(len(Y_female))
print(demographics_male.shape)
print(demographics_female.shape)


606
606
964
964
(606, 3)
(964, 3)


In [12]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
device = torch.device('cpu')
eval_metrics1 = np.zeros((skf.n_splits, 5))
eval_metrics2 = np.zeros((skf.n_splits, 5))
eval_metrics3 = np.zeros((skf.n_splits, 5))
eval_metrics4 = np.zeros((skf.n_splits, 5))

dataset = X_loaded
labels = Y_loaded
# demographics = demographics_female

In [13]:
dataset[0].shape

(116, 200)

In [14]:
# max_val_model = SkipConnModel((end - start + 4), 2, 3).to(device)

In [15]:
# Assuming skf is defined as an instance of StratifiedKFold
for n_fold, (train_val, test) in enumerate(skf.split(dataset, labels)):
    print('dataset = ',len(dataset))
    print('labels = ',len(labels))
    print('demogr = ',len(demographics))
    model = SkipConnModel((end - start + 3), 2,3).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)

    # Split dataset, labels, and demographics into train/validation and test sets
    train_val_dataset = [dataset[i] for i in train_val]
    test_dataset = [dataset[i] for i in test]
    train_val_labels = labels[train_val]
    test_labels = labels[test]
    train_val_demographics = demographics[train_val]
    test_demographics = demographics[test]

    # Create index array for train_val_dataset
    train_val_index = np.arange(len(train_val_dataset))

    # Split train_val into train and validation sets
    train_idx, val_idx, _, _ = train_test_split(
        train_val_index,
        train_val_labels,
        test_size=0.1,
        shuffle=True,
        stratify=train_val_labels, 
        random_state=seed
    )

    train_dataset = [train_val_dataset[i] for i in train_idx]
    val_dataset = [train_val_dataset[i] for i in val_idx]
    train_labels = [train_val_labels[i] for i in train_idx]
    val_labels = [train_val_labels[i] for i in val_idx]
    train_demographics = [train_val_demographics[i] for i in train_idx]
    val_demographics = [train_val_demographics[i] for i in val_idx]

    X_train_featgraph, X_train_adjgraph, Y_train, _ = create_graph_sliding_window_demographics(train_dataset, train_demographics, train_labels, start, end, region=True)
    X_val_featgraph, X_val_adjgraph, Y_val = create_graph_demographics(val_dataset, val_demographics, val_labels, start, end, region=True)
    X_test_featgraph, X_test_adjgraph, Y_test = create_graph_demographics(test_dataset, test_demographics, test_labels, start, end, region=True)

    X_train_datalist = to_tensor(X_train_featgraph, X_train_adjgraph, Y_train)
    X_val_datalist = to_tensor(X_val_featgraph, X_val_adjgraph, Y_val)
    X_test_datalist = to_tensor(X_test_featgraph, X_test_adjgraph, Y_test)
    
    if(len(X_train_datalist)%32==1):
        batch = 31
    else:
        batch = 32
        
    train_loader = DataLoader(X_train_datalist, batch_size=batch, shuffle=True, num_workers=0, worker_init_fn=seed_worker, generator=g)
    val_loader = DataLoader(X_val_datalist, batch_size=32, shuffle=True, num_workers=0, worker_init_fn=seed_worker, generator=g)
    test_loader = DataLoader(X_test_datalist, batch_size=32, shuffle=True, num_workers=0, worker_init_fn=seed_worker, generator=g)

    min_v_loss1 = np.inf
    best_val_acc1 = 0
    best_test_acc1 = 0
    best_test_f11 = 0
    best_test_sen1 = 0
    best_test_spe1 = 0
    best_test_auc1 = 0

    best_test_acc2 = 0
    best_test_f12 = 0
    best_test_sen2 = 0
    best_test_spe2 = 0
    best_test_auc2 = 0

    best_val_acc3 = 0
    best_test_acc3 = 0
    best_test_f13 = 0
    best_test_sen3 = 0
    best_test_spe3 = 0
    best_test_auc3 = 0

    best_val_auc4 = 0
    best_test_acc4 = 0
    best_test_f14 = 0
    best_test_sen4 = 0
    best_test_spe4 = 0
    best_test_auc4 = 0
    for epoch in range(50):
        _, _, _, _, t_loss = GCN_train(train_loader)
        val_sen, val_spe, val_acc,val_f1, val_auc, v_loss = GCN_test(val_loader)
        test_sen, test_spe, test_acc,test_f1, test_auc, _ = GCN_test(test_loader)
        # if epoch == 0:
        #     print('first = ',t_loss)
        # if epoch == 99:
        #     print('last = ',t_loss)

        
        #if np.isnan(val_sen) or np.isnan(val_spe) or np.isnan(val_acc) or np.isnan(val_f1) or np.isnan(val_auc):
        #    print(f"NaN found in validation metrics at epoch {epoch}")
        #    break

        #if np.isnan(test_sen) or np.isnan(test_spe) or np.isnan(test_acc) or np.isnan(test_f1) or np.isnan(test_auc):
        #    print(f"NaN found in test metrics at epoch {epoch}")
        #    break

        if min_v_loss1 > v_loss:
            min_v_loss1 = v_loss
            best_val_acc1 = val_acc
            best_test_f11 = test_f1
            best_test_sen1, best_test_spe1, best_test_acc1, best_test_auc1 = test_sen, test_spe, test_acc, test_auc
            # torch.save(model.state_dict(), 'SW_LANCET_results/model_min_vloss/best_model_%02i.pth' % (n_fold+1))
        
        if test_acc > best_test_acc2:  # Check if current test accuracy is better than the best recorded
            best_test_acc2 = test_acc
            best_test_f12 = test_f1
            best_test_sen2, best_test_spe2,best_test_auc2 = test_sen, test_spe, test_auc
            # torch.save(model.state_dict(), 'SW_LANCET_results/model_max_test_acc/best_model_%02i.pth' % (n_fold+1))
        
        if val_acc > best_val_acc3:
            best_val_acc3 = val_acc
            best_test_f13 = test_f1
            best_test_sen3, best_test_spe3, best_test_acc3, best_test_auc3 = test_sen, test_spe, test_acc, test_auc
            # max_val_model = model
            # Create the directory if it does not exist
            os.makedirs('Analysis/Male_Female_Bregions', exist_ok=True)
            torch.save(model.state_dict(), f'Analysis/Male_Female_Bregions/model_female_2.pth')
        
        if val_auc > best_val_auc4:
            best_val_auc4 = val_auc
            best_test_f14 = test_f1
            best_test_sen4, best_test_spe4, best_test_acc4, best_test_auc4 = test_sen, test_spe, test_acc, test_auc
            
        print('CV: {:03d}, Epoch: {:03d}, Val Loss: {:.5f}, Val ACC: {:.5f},Val AUC: {:.5f}, Test ACC: {:.5f}, Test F1: {:.5f}, TEST SPE: {:.5f}, '
                  'TEST SEN: {:.5f}, TEST AUC: {:.5f}'.format(n_fold +1, epoch + 1, v_loss, val_acc, val_auc, test_acc,test_f1,
                                        test_spe,test_sen, test_auc))
    
            
    eval_metrics1[n_fold, 0] = best_test_sen1
    eval_metrics1[n_fold, 1] = best_test_spe1
    eval_metrics1[n_fold, 2] = best_test_acc1
    eval_metrics1[n_fold, 3] = best_test_f11
    eval_metrics1[n_fold, 4] = best_test_auc1

    eval_metrics2[n_fold, 0] = best_test_sen2
    eval_metrics2[n_fold, 1] = best_test_spe2
    eval_metrics2[n_fold, 2] = best_test_acc2
    eval_metrics2[n_fold, 3] = best_test_f12
    eval_metrics2[n_fold, 4] = best_test_auc2

    eval_metrics3[n_fold, 0] = best_test_sen3
    eval_metrics3[n_fold, 1] = best_test_spe3
    eval_metrics3[n_fold, 2] = best_test_acc3
    eval_metrics3[n_fold, 3] = best_test_f13
    eval_metrics3[n_fold, 4] = best_test_auc3
        
    eval_metrics4[n_fold, 0] = best_test_sen4
    eval_metrics4[n_fold, 1] = best_test_spe4
    eval_metrics4[n_fold, 2] = best_test_acc4
    eval_metrics4[n_fold, 3] = best_test_f14
    eval_metrics4[n_fold, 4] = best_test_auc4

    break


dataset =  1570
labels =  1570
demogr =  1570
CV: 001, Epoch: 001, Val Loss: 16.74726, Val ACC: 0.71831,Val AUC: 0.76948, Test ACC: 0.61783, Test F1: 0.68085, TEST SPE: 0.44000, TEST SEN: 0.78049, TEST AUC: 0.64862
CV: 001, Epoch: 002, Val Loss: 16.11673, Val ACC: 0.73239,Val AUC: 0.79452, Test ACC: 0.63694, Test F1: 0.69841, TEST SPE: 0.45333, TEST SEN: 0.80488, TEST AUC: 0.67122
CV: 001, Epoch: 003, Val Loss: 15.10120, Val ACC: 0.76056,Val AUC: 0.82492, Test ACC: 0.63057, Test F1: 0.68132, TEST SPE: 0.49333, TEST SEN: 0.75610, TEST AUC: 0.68520
CV: 001, Epoch: 004, Val Loss: 16.05330, Val ACC: 0.69014,Val AUC: 0.80743, Test ACC: 0.63057, Test F1: 0.71000, TEST SPE: 0.37333, TEST SEN: 0.86585, TEST AUC: 0.68016
CV: 001, Epoch: 005, Val Loss: 15.02713, Val ACC: 0.76761,Val AUC: 0.81558, Test ACC: 0.63694, Test F1: 0.66667, TEST SPE: 0.57333, TEST SEN: 0.69512, TEST AUC: 0.68260
CV: 001, Epoch: 006, Val Loss: 16.56324, Val ACC: 0.66197,Val AUC: 0.81995, Test ACC: 0.58599, Test F1: 0.694

In [16]:

print("Corresponding to minimum val loss")
eval_df1 = pd.DataFrame(eval_metrics1)
eval_df1.columns = ['SEN', 'SPE', 'ACC','F1', 'AUC-ROC']
eval_df1.index = ['Fold_%02i' % (i + 1) for i in range(skf.n_splits)]
print(eval_df1)
print('Average Sensitivity: %.4f±%.4f' % (eval_metrics1[:, 0].mean(), eval_metrics1[:, 0].std()))
print('Average Specificity: %.4f±%.4f' % (eval_metrics1[:, 1].mean(), eval_metrics1[:, 1].std()))
print('Average Accuracy: %.4f±%.4f' % (eval_metrics1[:, 2].mean(), eval_metrics1[:, 2].std()))
print('Average F1: %.4f±%.4f' % (eval_metrics1[:, 3].mean(), eval_metrics1[:, 3].std()))
print('Average AUC-ROC: %.4f±%.4f' % (eval_metrics1[:, 4].mean(), eval_metrics1[:, 4].std()))


print("\nCorresponding to best test acc")
eval_df2 = pd.DataFrame(eval_metrics2)
eval_df2.columns = ['SEN', 'SPE', 'ACC','F1', 'AUC-ROC']
eval_df2.index = ['Fold_%02i' % (i + 1) for i in range(skf.n_splits)]
print(eval_df2)
print('Average Sensitivity: %.4f±%.4f' % (eval_metrics2[:, 0].mean(), eval_metrics2[:, 0].std()))
print('Average Specificity: %.4f±%.4f' % (eval_metrics2[:, 1].mean(), eval_metrics2[:, 1].std()))
print('Average Accuracy: %.4f±%.4f' % (eval_metrics2[:, 2].mean(), eval_metrics2[:, 2].std()))
print('Average F1: %.4f±%.4f' % (eval_metrics2[:, 3].mean(), eval_metrics2[:, 3].std()))
print('Average AUC-ROC: %.4f±%.4f' % (eval_metrics2[:, 4].mean(), eval_metrics2[:, 4].std()))

print("\nCorresponding to maxm val acc")
eval_df3 = pd.DataFrame(eval_metrics3)
eval_df3.columns = ['SEN', 'SPE', 'ACC','F1', 'AUC-ROC']
eval_df3.index = ['Fold_%02i' % (i + 1) for i in range(skf.n_splits)]
print(eval_df3)
print('Average Sensitivity: %.4f±%.4f' % (eval_metrics3[:, 0].mean(), eval_metrics3[:, 0].std()))
print('Average Specificity: %.4f±%.4f' % (eval_metrics3[:, 1].mean(), eval_metrics3[:, 1].std()))
print('Average Accuracy: %.4f±%.4f' % (eval_metrics3[:, 2].mean(), eval_metrics3[:, 2].std()))
print('Average F1: %.4f±%.4f' % (eval_metrics3[:, 3].mean(), eval_metrics3[:, 3].std()))
print('Average AUC-ROC: %.4f±%.4f' % (eval_metrics3[:, 4].mean(), eval_metrics3[:, 4].std()))

print("\nCorresponding to maxm val AUC")
eval_df4 = pd.DataFrame(eval_metrics4)
eval_df4.columns = ['SEN', 'SPE', 'ACC','F1', 'AUC-ROC']
eval_df4.index = ['Fold_%02i' % (i + 1) for i in range(skf.n_splits)]
print(eval_df4)
print('Average Sensitivity: %.4f±%.4f' % (eval_metrics4[:, 0].mean(), eval_metrics4[:, 0].std()))
print('Average Specificity: %.4f±%.4f' % (eval_metrics4[:, 1].mean(), eval_metrics4[:, 1].std()))
print('Average Accuracy: %.4f±%.4f' % (eval_metrics4[:, 2].mean(), eval_metrics4[:, 2].std()))
print('Average F1: %.4f±%.4f' % (eval_metrics4[:, 3].mean(), eval_metrics4[:, 3].std()))
print('Average AUC-ROC: %.4f±%.4f' % (eval_metrics4[:, 4].mean(), eval_metrics4[:, 4].std()))

Corresponding to minimum val loss
              SEN       SPE       ACC        F1   AUC-ROC
Fold_01  0.695122  0.573333  0.636943  0.666667  0.682602
Fold_02  0.000000  0.000000  0.000000  0.000000  0.000000
Fold_03  0.000000  0.000000  0.000000  0.000000  0.000000
Fold_04  0.000000  0.000000  0.000000  0.000000  0.000000
Fold_05  0.000000  0.000000  0.000000  0.000000  0.000000
Fold_06  0.000000  0.000000  0.000000  0.000000  0.000000
Fold_07  0.000000  0.000000  0.000000  0.000000  0.000000
Fold_08  0.000000  0.000000  0.000000  0.000000  0.000000
Fold_09  0.000000  0.000000  0.000000  0.000000  0.000000
Fold_10  0.000000  0.000000  0.000000  0.000000  0.000000
Average Sensitivity: 0.0695±0.2085
Average Specificity: 0.0573±0.1720
Average Accuracy: 0.0637±0.1911
Average F1: 0.0667±0.2000
Average AUC-ROC: 0.0683±0.2048

Corresponding to best test acc
              SEN       SPE      ACC        F1   AUC-ROC
Fold_01  0.731707  0.586667  0.66242  0.693642  0.678049
Fold_02  0.000000  0.00

In [17]:
def calculate_grad_cam(model, data):
    data = data.to(device)
    model.zero_grad()
    data.x.requires_grad = True  # Set requires_grad to True for computing gradients

    # Forward pass
    output, _ = model(data)  # Get the prediction scores

    # Apply softmax to calculate scores (probabilities)
    scores = F.softmax(output, dim=1)
    print(scores.shape)
    print(scores[0])

    # Extract MDD scores
    scores_mdd = scores[:, 1]  # Assuming the second column corresponds to MDD
    print('scores = ',scores_mdd.shape)

    # Backward pass to compute gradients with respect to data.x
    scores_mdd.backward(torch.ones_like(scores_mdd))  # Calculate gradients

    # Retrieve the gradients of scores_mdd with respect to data.x
    gradients = data.x.grad
    print('gradients = ',gradients.shape)  # Check the shape of gradients

    # Calculate Grad-CAM values by element-wise multiplying gradients and data.x
    grad_cam = gradients * data.x
    print('data.x = ',data.x.shape)  # Check the shape of Grad-CAM values
    print('gradcam = ',grad_cam.shape)  # Check the shape of Grad-CAM values

    # Sum the Grad-CAM values for each brain region (column-wise sum)
    # contribution = grad_cam.sum(dim=0)
    contribution = torch.mean(grad_cam, dim=1)
    
    print('contribution = ',contribution.shape)  # Check the shape of Grad-CAM values

    return scores, grad_cam, contribution

In [18]:
val_loader2 = DataLoader(X_val_datalist, batch_size=1, shuffle=True, num_workers=0, worker_init_fn=seed_worker, generator=g)
test_loader2 = DataLoader(X_test_datalist, batch_size=1, shuffle=True, num_workers=0, worker_init_fn=seed_worker, generator=g)
print(len(X_val_datalist))
print(X_test_datalist[0])

best_test_model = SkipConnModel((end - start + 3), 2, 3).to(device)
state_dict = torch.load('Analysis/Male_Female_Bregions/model_female_2.pth')

# Load the state dictionary into the model
best_test_model.load_state_dict(state_dict)

#Evaluate data
best_test_model.eval()
# Use the function
# Initialize an empty list to store contributions from each batch
contributions_list = []

# Assuming val_loader2 is your validation data loader
for batch in val_loader2:
    scores, grad_cam, contribution = calculate_grad_cam(best_test_model, batch)
    
    # Move contribution tensor to CPU before appending to list
    contribution = contribution.cpu()   
    
    # Append contribution to the list
    contributions_list.append(contribution)

# Convert contributions_list to a numpy array after moving all tensors to CPU
contributions_array = np.array([item.detach().numpy() for item in contributions_list])
print('contri array = ', contributions_array.shape)

# Calculate the mean across all contributions
mean_contribution = np.mean(contributions_array, axis=0)
print('contri mean = ', mean_contribution.shape)

# Print or use the mean_contribution as needed
print("Mean contribution shape:", mean_contribution.shape)
# print("Mean contribution:", mean_contribution)   
    

           

142
Data(x=[116, 119], edge_index=[2, 1336], edge_attr=[1336], y=[1])
torch.Size([1, 2])
tensor([0.7831, 0.2169], grad_fn=<SelectBackward0>)
scores =  torch.Size([1])
gradients =  torch.Size([116, 119])
data.x =  torch.Size([116, 119])
gradcam =  torch.Size([116, 119])
contribution =  torch.Size([116])
torch.Size([1, 2])
tensor([0.3336, 0.6664], grad_fn=<SelectBackward0>)
scores =  torch.Size([1])
gradients =  torch.Size([116, 119])
data.x =  torch.Size([116, 119])
gradcam =  torch.Size([116, 119])
contribution =  torch.Size([116])
torch.Size([1, 2])
tensor([0.2313, 0.7687], grad_fn=<SelectBackward0>)
scores =  torch.Size([1])
gradients =  torch.Size([116, 119])
data.x =  torch.Size([116, 119])
gradcam =  torch.Size([116, 119])
contribution =  torch.Size([116])
torch.Size([1, 2])
tensor([0.9044, 0.0956], grad_fn=<SelectBackward0>)
scores =  torch.Size([1])
gradients =  torch.Size([116, 119])
data.x =  torch.Size([116, 119])
gradcam =  torch.Size([116, 119])
contribution =  torch.Size([

In [19]:
mean_contribution

array([-1.63808028e-04, -2.27638084e-04,  4.52868517e-05, -9.02492029e-05,
       -1.02553087e-04, -3.84232466e-04, -1.32203655e-04, -1.37665513e-04,
       -2.96640268e-04, -2.93525140e-04, -2.33579878e-04, -1.95891698e-04,
       -2.37408691e-04, -1.70582876e-04, -2.19712965e-04, -1.79993251e-04,
       -1.74765359e-04,  5.17941189e-05, -6.00122366e-05,  3.83020961e-05,
       -1.26342347e-04, -1.25104460e-04, -1.09362125e-04, -8.19016786e-05,
        2.36086446e-04,  1.88090286e-04,  5.48485514e-06, -3.49083675e-05,
       -1.47977014e-04, -1.29998065e-04,  1.22389843e-04, -6.57819764e-05,
        1.16068150e-04,  4.18350055e-06,  1.75450737e-06,  1.09010643e-05,
       -3.43017746e-05, -8.94023651e-06, -3.81912359e-05,  2.27526198e-05,
       -3.13020391e-05,  3.41469568e-05, -1.50485284e-04, -1.61818243e-04,
       -1.88093996e-04, -1.86147954e-04, -6.90087327e-05, -1.19757169e-04,
       -1.72357250e-04, -1.69277875e-04, -7.95489177e-05,  3.46225679e-05,
       -7.62178897e-05, -

In [20]:

#Assuming 'avg_grad_cam' is your tensor of average Grad-CAM values
# Sort mean_contribution and get sorted indices
sorted_indices = np.argsort(-mean_contribution)

# print("Sorted Grad-CAM values:", sorted_grad_cam_values)
print("Indices of sorted Grad-CAM values:", sorted_indices)

Indices of sorted Grad-CAM values: [ 66  24  25 110  67  30  32  79 112  62  54  68  17  58   2 107  19  51
  41  69  90  39  35  26  33  34  37  78  99  55  40  97  36  27  38  95
  56  87 101 106  85  18  64  96  75  31 108 113  46  52  50  53  23  63
   3  80 105  98  88  94   4  59  22  74 115  57  91  81  72 111  47  77
  82  21  20  76  29 103   6   7  93  73  28 109  42  89  92  60 114 104
  43   0  84  49  13  48  16  15  45  44  86  11 100  65  71  14   1  10
  12  61  83 102  70   9   8   5]
